In [ ]:
%pip install numpy
%pip install -r requirements.txt

In [1]:
# Data Ingestion
from langchain_community.document_loaders import TextLoader

loader = TextLoader("example.txt")
documents = loader.load()
documents

[Document(metadata={'source': 'example.txt'}, page_content='Commercial law in Sri Lanka governs the legal principles that regulate business and commercial transactions within the country. Rooted in Roman-Dutch law and influenced by English common law, Sri Lankaâ€™s commercial legal system covers a wide range of areas including contracts, trade, banking, insurance, partnerships, and company law. The Sale of Goods Ordinance, the Companies Act No. 7 of 2007, and the Partnership Ordinance are key legislative instruments that guide business practices. These laws provide the framework for fair and transparent transactions, ensuring that the rights and obligations of all parties involved in commercial dealings are legally protected.\n\nIn recent years, Sri Lanka has made efforts to modernize its commercial legal landscape to align with international standards and encourage foreign investment. The establishment of specialized commercial courts, digital company registration services, and altern

In [2]:
# PDF Ingestion
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("detailed.pdf")
pdf = loader.load()
pdf

[Document(metadata={'producer': 'xdvipdfmx (20220710)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-05-10T05:59:41+00:00', 'source': 'detailed.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Detailed Brief on Commercial Law: Contracts\nMay 10, 2025\nIn commercial law, contracts serve as the foundation for enforceable agreements\nthat facilitate business transactions. A valid contract requires several essential\nelements, as outlined in various legal frameworks, including South African com-\nmon law and statutes like theConsumer Protection Act 68 of 2008. These elements\ninclude an agreement between parties, evidenced by an offer and acceptance,\nand an intention to create legally binding obligations. A mutual understand-\ning, or “meeting of the minds,” ensures both parties comprehend the contract’s\nterms. Additionally, parties must have contractual capacity—being of legal age\nand sound mind—and the contract’s purpose must be lawful, with terms capa-\

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
# Chunking Data

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
)

chunked_pdf = text_splitter.split_documents(pdf)
chunked_pdf

[Document(metadata={'producer': 'xdvipdfmx (20220710)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-05-10T05:59:41+00:00', 'source': 'detailed.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Detailed Brief on Commercial Law: Contracts\nMay 10, 2025\nIn commercial law, contracts serve as the foundation for enforceable agreements\nthat facilitate business transactions. A valid contract requires several essential\nelements, as outlined in various legal frameworks, including South African com-'),
 Document(metadata={'producer': 'xdvipdfmx (20220710)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-05-10T05:59:41+00:00', 'source': 'detailed.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='elements, as outlined in various legal frameworks, including South African com-\nmon law and statutes like theConsumer Protection Act 68 of 2008. These elements\ninclude an agreement between parties, evidenced by an offer and acceptance,'),
 Doc

In [5]:
# Vector Embedding and Vector Store

from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

vector_store = Chroma.from_documents(
    chunked_pdf,
    embeddings,
    collection_name="example_collection"
)

C:\Users\pasin\AppData\Local\Temp\ipykernel_25184\1540851800.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [6]:
# Vector database Querying

query = "What Are the Essential Elements of a Valid Contract?"
results = vector_store.similarity_search(query)
results

[Document(metadata={'source': 'detailed.pdf', 'creationdate': '2025-05-10T05:59:41+00:00', 'producer': 'xdvipdfmx (20220710)', 'total_pages': 2, 'page_label': '1', 'creator': 'LaTeX with hyperref', 'page': 0}, page_content='These principles align with global standards, such as those in theUNIDROIT Prin-\nciples of International Commercial Contracts(2016), which emphasize consensus\nand legality (UNIDROIT, 2016).\nNot all contracts require written form; they can be oral or written, depending on'),
 Document(metadata={'source': 'detailed.pdf', 'page_label': '1', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20220710)', 'creationdate': '2025-05-10T05:59:41+00:00', 'page': 0, 'total_pages': 2}, page_content='and sound mind—and the contract’s purpose must be lawful, with terms capa-\nble of performance. For instance, a contract to sell illegal goods would be void.\nThese principles align with global standards, such as those in theUNIDROIT Prin-'),
 Document(metadata={'total_page

In [7]:
# FAISS database

from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

faiss_vector_store = FAISS.from_documents(
    chunked_pdf,
    embeddings
)

In [8]:
# Vector database Querying

query = "What Are the Essential Elements of a Valid Contract?"
results = faiss_vector_store.similarity_search(query)
results

[Document(id='46c993eb-2416-476d-9fef-a605125fa8ed', metadata={'producer': 'xdvipdfmx (20220710)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-05-10T05:59:41+00:00', 'source': 'detailed.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='These principles align with global standards, such as those in theUNIDROIT Prin-\nciples of International Commercial Contracts(2016), which emphasize consensus\nand legality (UNIDROIT, 2016).\nNot all contracts require written form; they can be oral or written, depending on'),
 Document(id='2b475111-9525-407a-ae9f-4e22688fb677', metadata={'producer': 'xdvipdfmx (20220710)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-05-10T05:59:41+00:00', 'source': 'detailed.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='and sound mind—and the contract’s purpose must be lawful, with terms capa-\nble of performance. For instance, a contract to sell illegal goods would be void.\nThese principles align with g

In [9]:
# from langchain_community.llms import Ollama

# llm = Ollama(model=llama3)
# llm

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

response = llm.invoke("What's the capital of Sri Lanka?")
print(response.content)

Sri Lanka has two capitals:

*   **Colombo** is the commercial capital.
*   **Sri Jayawardenepura Kotte** is the administrative capital.


In [39]:
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-exp', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002298299AFD0>, default_metadata=())

In [40]:
# Design a Prompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
                                            Answer the following question based only on the provided context.
                                            Think step by step before providing a detailed answer. 
                                            Explain the answer in a way that is easy to understand using only the provided context.
                                            I will tip you $1000 if the user finds the answer helpful. 
                                            Do NOT include phrases like "Based on the context" or "Here's the answer".
                                            ONLY return the answer.
                                            <context>
                                            {context}
                                            </context>
                                            Question: {input}
                                        """)

In [41]:
# Create Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

doc_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

In [42]:
retriever = faiss_vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000229EACE6A50>, search_kwargs={})

In [43]:
# Retrieval Chain

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, doc_chain)

In [ ]:
response = retrieval_chain.invoke({
    "input": "What Are the Essential Elements of a Valid Contract?"
})

response['answer']

'The essential elements of a valid contract include an agreement between parties, evidenced by an offer and acceptance, and an intention to create legally binding obligations. A mutual understanding, or “meeting of the minds,” ensures both parties comprehend the contract’s terms. Parties entering the contract must have the capacity to contract—legal age and sound mind—and the contract’s purpose must be lawful, with terms capable of performance.'

In [45]:
response = retrieval_chain.invoke({
    "input": "Do All Contracts Need to Be in Writing?"
})

response['answer']

'No, not all contracts require written form. They can be oral or written.'

In [46]:
response = retrieval_chain.invoke({
    "input": "What Are the Remedies Available for Breach of Contract?"
})

response['answer']

'Remedies for breach aim to restore the aggrieved party’s position. Specific performance, a court-ordered fulfillment of obligations, is common when monetary compensation is inadequate. Also termination of the contract if the breach is material is a remedy.'